In [25]:
file_path = r"C:\Users\MY PC\Downloads\Global_Superstore2.csv"  # Update path to your dataset
df = pd.read_csv(file_path, encoding="ISO-8859-1")
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,05-11-2013,06-11-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


<h3>

the dataset is clean

In [2]:
# ! pip install dash
# !pip install jupyter_dash
# !pip install plotly.express
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.express as px
###
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
###
import random

# Step 1: Load the dataset
file_path = r"C:\Users\MY PC\Downloads\Global_Superstore2.csv"  # Update path to your dataset
df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Step 2: Data Cleaning
df.drop(["Row ID", "Postal Code"], axis=1, inplace=True)  # Dropping irrelevant columns
df.dropna(inplace=True)  # Remove rows with missing values

df["Order Date"] = pd.to_datetime(df["Order Date"], dayfirst=True)  # Parse dates as day-month-year
df["Ship Date"] = pd.to_datetime(df["Ship Date"], dayfirst=True)  # Parse dates as day-month-year

df["Profit Margin"] = (df["Profit"] / df["Sales"]) * 100  # Add Profit Margin column

# Step 3: Initialize Dash app
app = dash.Dash(__name__)
app.title = "Global Superstore Dashboard"  # Browser tab title

# Global Chart Style (Dark Theme)
chart_layout = {
    "paper_bgcolor": "#2C3E50",  # Chart background color
    "plot_bgcolor": "#34495E",  # Plot area background color
    "font": {"color": "white"},  # Font color
    "title": {"x": 0.5},  # Center the title
}

# Layout
app.layout = html.Div(
    style={
        "background-image": "url('https://www.transparenttextures.com/patterns/diamond-upholstery.png')",  # Subtle pattern
        "background-color": "#2C3E50",  # Dark blue background
        "color": "white",  # Text color
        "font-family": "Arial, sans-serif",  # Font style
        "min-height": "100vh",  # Full viewport height
        "padding": "20px",
    },
    children=[
        # Header with custom styling
        html.Div(
            style={
                "background-color": "#34495E",  # Darker blue for the header
                "padding": "20px",
                "border-radius": "10px",
                "text-align": "center",
                "box-shadow": "0px 4px 10px rgba(0,0,0,0.5)",  # Shadow for header
            },
            children=[
                html.H1(
                    "Global Superstore Dashboard",
                    style={"color": "white", "font-size": "36px", "margin": "0"},
                ),
                html.P(
                    "Analyze sales and profit data across regions, categories, and time",
                    style={"color": "#D5D8DC", "font-size": "18px", "margin-top": "10px"},
                ),
            ],
        ),

        # Dropdown menu for Region Filter
        html.Div(
            style={"padding": "20px", "text-align": "center"},
            children=[
                dcc.Dropdown(
                    id="region-filter",
                    options=[
                        {"label": region, "value": region}
                        for region in df["Region"].unique()
                    ],
                    value="Central",
                    placeholder="Select a Region",
                    style={
                        "width": "50%",
                        "margin": "0 auto",
                        "background-color": "#ECF0F1",
                        "color": "black",
                        "border-radius": "5px",
                        "padding": "10px",
                    },
                )
            ],
        ),

        # Sales Bar Chart
        html.Div(
            style={"margin-top": "20px"},
            children=[dcc.Graph(id="sales-bar-chart")],
        ),

        # Profit Line Chart
        html.Div(
            style={"margin-top": "20px"},
            children=[dcc.Graph(id="profit-line-chart")],
        ),

        # Category Pie Chart
        html.Div(
            style={"margin-top": "20px"},
            children=[dcc.Graph(id="category-pie-chart")],
        ),
    ],
)

# Callback for interactivity
@app.callback(
    [
        Output("sales-bar-chart", "figure"),
        Output("profit-line-chart", "figure"),
        Output("category-pie-chart", "figure"),
    ],
    [Input("region-filter", "value")],
)
def update_charts(selected_region):
    """
    Update graphs based on the selected region.
    """
    # Filter data by the selected region
    filtered_df = df[df["Region"] == selected_region]

    # Sales Bar Chart: Sales by Sub-Category
    sales_bar = px.bar(
        filtered_df,
        x="Sub-Category",
        y="Sales",
        title=f"Sales by Sub-Category in {selected_region}",
        color="Sub-Category",
        color_discrete_sequence=px.colors.qualitative.Bold,  # Bold color scheme
    )
    sales_bar.update_layout(**chart_layout)  # Apply the dark theme

    # Profit Line Chart: Profit over Time
    profit_line = px.line(
        filtered_df,
        x="Order Date",
        y="Profit",
        title=f"Profit Over Time in {selected_region}",
        color="Category",
        color_discrete_sequence=px.colors.qualitative.Set1,  # Set1 color scheme
    )
    profit_line.update_layout(**chart_layout)  # Apply the dark theme

    # Category Pie Chart: Sales Distribution by Category
    category_pie = px.pie(
        filtered_df,
        names="Category",
        values="Sales",
        title=f"Sales Distribution by Category in {selected_region}",
        color_discrete_sequence=px.colors.sequential.RdBu,  # Sequential red-blue color scheme
    )
    category_pie.update_layout(**chart_layout)  # Apply the dark theme

    return sales_bar, profit_line, category_pie


# Function to get an available port
def get_available_port():
    while True:
        port = random.randint(8050, 8090)  # Random port selection
        try:
            # Try running the app with the chosen port
            app.run_server(debug=True, port=port, mode="inline", use_reloader=False)
            return port  # Return the available port if successful
        except OSError:
            pass  # Port is in use, try another one

# Run the server and display the port
if __name__ == "__main__":
    # Get an available port
    port = get_available_port()
    print(f"App is running on port: {port}")
    print(f"You can manually open it in the browser at http://127.0.0.1:{port}")


C:\Users\MY PC\AppData\Local\Temp\ipykernel_10812\1497992065.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\MY PC\AppData\Local\Temp\ipykernel_10812\1497992065.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


App is running on port: 8059
You can manually open it in the browser at http://127.0.0.1:8059
